<a href="https://colab.research.google.com/github/kimdw92/kaggle_study/blob/main/surprise_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 파이썬 추천 시스템 패키지 - Surprise
Surprise는 추천 시스템을 위한 파이썬 기반의 사이킷런과 유사한 API와 프레임워크를 제공한다.

In [2]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 4.6 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617739 sha256=b97e217d0cbb3d95be70828f8b9d8789383131986aa72d24e573ab69eeb52cd0
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [3]:
import surprise 

print(surprise.__version__)

1.1.1


## Surprise를 이용한 추천 시스템 구축 예제

SVD 행렬 분해를 통한 잠재 요인 CF를 수행해보자

In [4]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

Surprise는 MovieLens 사이트에서 제공하는 과거 버전의 데이터 세트를 가져오는 API를 제공한다.  
'ml-100k'(10만 개 평점 데이터) 또는 'ml-1m'(100만 개 평점 데이터) 등 을 제공한다.

In [5]:
data = Dataset.load_builtin('ml-100k') 
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


Surprise에 사용자-아이템 평점 데이터를 적용할 때 주의해야 할 점은 row 레벨의 데이터를 그대로 사용해야 한다는 것이다. 판다스의 pivot_table 함수로 굳이 사용자-row, 아이템-col 형태로 바꾸어줄 필요가 없다.

In [13]:
algo = SVD()
algo.fit(trainset)

Surpirse에서 추천을 예측하는 메소드는 test()와 predict() 두 개이다.  
test()는 전체 평점 데이터에 대해서 추천을 예측한다.

In [14]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5430079384862334, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.8215663073867616, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.1132783209647865, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.618968369243448, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.519361060607423, details={'was_impossible': False})]

SVD 알고리즘 객체의 test 메서드의 호출 결과는 파이썬 리스트이며 내부에는 Prediction 객체를 가진다.

In [15]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.5430079384862334),
 ('882', '291', 3.8215663073867616),
 ('535', '507', 4.1132783209647865)]

predict()는 개별 사용자의 아이템에 대한 추천 평점을 예측해 준다.

In [16]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.58   {'was_impossible': False}


rmse() 함수는 추천 시스템의 성능 평가 정보를 제공한다. 추천 예측 평점과 실제 평점과의 차이

In [17]:
accuracy.rmse(predictions)

RMSE: 0.9475


0.9475032491321169

## Surprise를 이용한 개인화 영화 추천 시스템 구축

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import pandas as pd
import numpy as np

movies = pd.read_csv('/content/drive/MyDrive/datasets/ml-latest-small/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/datasets/ml-latest-small/ratings.csv')

Surprise는 데이터 세트를 train_test_split()을 이용해 내부에서 사용하는 TrainSet 클래스 객체로 변환하지 않으면 fit()을 사용할 수가 없다.  
DatasetAutoFolds 클래스를 이용해 데이터 세트 전체를 학습 데이터로 사용할 수 있다. (상당히 번거롭네...?)

In [21]:
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('/content/drive/MyDrive/datasets/ml-latest-small/ratings_noh.csv', index=False, header=False)

In [23]:
from surprise.dataset import DatasetAutoFolds
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='/content/drive/MyDrive/datasets/ml-latest-small/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

SVD를 이용한 학습

In [24]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

### movieId 42인 영화에 대해서 userId 9 사용자의 추천 예상 평점

In [25]:
# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [26]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


사용자 9가 평점을 매기지 않은 영화들 중에서 추천을 해보자

In [27]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [28]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
